<a href="https://colab.research.google.com/github/valfeclone/ML-Layani/blob/main/TweepyCursor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tweepy


# API Tweepy

In [2]:
import tweepy
import json
import csv
import pandas as pd
from datetime import timedelta

# Authenticate to Twitter
auth = tweepy.OAuthHandler("KD8NW5hyiGCIhwp4JDJ98pgXh", "SQeokZiYEb5nqWiRMzwRIWF2QQyeBANeHvyfYg3BjenCp2T5vV")
auth.set_access_token("93126766-LZ166ZBBYttZ4JM5WKg9cLLBq0eoJyA0QJwq7l30V", "BlyNL7pReqLDQx0uWjNkMeXlg4w2j2U7cFFpPE1ATy29X")
api = tweepy.API(auth)

Collecting Tweets

In [9]:
import datetime

def import_tweet():
  cursor = tweepy.Cursor(api.search, q=('jalan bolong'), lang="id", tweet_mode="extended").items(450)
  csvFile = open('jokowi gimana.csv', 'w')
  csvWriter = csv.writer(csvFile)
  csvWriter.writerow(["Timestamp", "ID", "Username", "Text"])
  for tweet in cursor:
    now = datetime.datetime.now()
    remain = now - tweet.created_at
    secs = remain.total_seconds()
    # print(secs)
    # print(tweet)
    # print(secs)
    if (secs < 7200):
      if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
        print(tweet)
        # c.append(tweet.created_at + timedelta(hours=7))
        # i.append(tweet.id)
        # u.append(tweet.user.name)
        # t.append(tweet.full_text)
        # l.append(tweet.location)
        tweets = [tweet.created_at + timedelta(hours=7),tweet.id,tweet.user.screen_name,tweet.full_text]
        csvWriter.writerow(tweets)

  csvFile.close()
  csvFile = open('jokowi gimana.csv', 'a')
  csvWriter = csv.writer(csvFile)
  cursor = tweepy.Cursor(api.search, q=('air mati'), lang="id", tweet_mode="extended").items(450)
  for tweet in cursor:
    now = datetime.datetime.now()
    remain = now - tweet.created_at
    secs = remain.total_seconds()
    # print(secs)
    # print(tweet)
    # print(secs)
    if (secs < 7200):
      if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
        print(tweet)
        # c.append(tweet.created_at + timedelta(hours=7))
        # i.append(tweet.id)
        # u.append(tweet.user.name)
        # t.append(tweet.full_text)
        # l.append(tweet.location)
        tweets = [tweet.created_at + timedelta(hours=7),tweet.id,tweet.user.screen_name,tweet.full_text]
        csvWriter.writerow(tweets)


# Predicting

In [4]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import string
import nltk
import random
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from keras.models import load_model

def lstm_model(new_dataset):
  dataTweet = pd.read_csv("CleanDataset.csv")
  new_dataTweet = pd.read_csv(new_dataset)

  def remove_punctuation(text):
    remover = str.maketrans("", "", string.punctuation)
    return text.translate(remover)

  def casefolding(review):
      review = review.lower()
      return review

  def remove_stopwords(text):
    sw = set(stopwords.words("indonesian"))
    words = [word.lower() for word in text.split() if word.lower() not in sw]
    return " ".join(words)

  dataTweet["Text"] = dataTweet.Text.map(remove_punctuation)
  dataTweet["Text"] = dataTweet.Text.map(casefolding)
  dataTweet["Text"] = dataTweet.Text.map(remove_stopwords)

  new_dataTweet["Text"] = new_dataTweet.Text.map(remove_punctuation)
  new_dataTweet["Text"] = new_dataTweet.Text.map(casefolding)
  new_dataTweet["Text"] = new_dataTweet.Text.map(remove_stopwords)

  def words_counter(text_column):
    count = Counter()
    for text in text_column.values:
      for word in text.split():
        count[word] += 1
    return count

  total_unique = len(words_counter(dataTweet.Text))
  new_total_unique = len(words_counter(new_dataTweet.Text))

  def split_data(dataset):
  	dataset = dataset.sample(frac=1, random_state=42)
  	train_data, val_data = dataset[:80], dataset[80:]
  	train_tweets = train_data.Text.to_numpy()
  	train_labels = train_data.Label.to_numpy()
  	val_tweets = val_data.Text.to_numpy()
  	val_labels = val_data.Label.to_numpy()
  	return train_data, train_tweets, train_labels, val_data, val_tweets, val_labels

  train_data, train_tweets, train_labels, val_data, val_tweets, val_labels = split_data(dataTweet)
  test_tweets = new_dataTweet.Text.to_numpy()

  def embedding(text):
    tokenizer = Tokenizer(num_words = total_unique)
    tokenizer.fit_on_texts(text)
    word_index = tokenizer.word_index
    text_sequences = tokenizer.texts_to_sequences(text)
    return text_sequences

  def padding(text_seq):
    text_padded = pad_sequences(text_seq, maxlen=40, padding="post", truncating="post")
    return text_padded

  train_sequences = embedding(train_tweets)
  train_padded = padding(train_sequences)
  val_sequences = embedding(val_tweets)
  val_padded = padding(val_sequences)
  test_sequences = embedding(test_tweets)
  test_padded = padding(test_sequences)

  model = keras.models.Sequential()
  model.add(layers.Embedding(total_unique, 32, input_length=40))
  model.add(layers.LSTM(64, dropout=0.1))
  model.add(layers.Dense(1, activation='sigmoid'))

  loss = keras.losses.BinaryCrossentropy(from_logits=False)
  optim = keras.optimizers.Adam(learning_rate=0.001)
  metrics = ["accuracy"]

  model.compile(loss=loss, optimizer=optim, metrics=metrics)

  model.fit(train_padded, train_labels, epochs=20, validation_data=(val_padded, val_labels), verbose=False)

  predictions = model.predict(test_padded)
  predictions = [1 if p > 0.5 else 0 for p in predictions]
	
  # csvFile2 = open('prabowo gimana.csv', 'w')
  # csvWriter2 = csv.writer(csvFile2)
  # csvWriter2.writerow(["Timestamp", "ID", "Username", "Text"])
  
  complaint = []
  new_complaints = {}
  counter = 0
  for i in range(len(predictions)):
    if predictions[i] == 1:
      complaint.append(new_dataTweet["Text"][i])
      new_complaints[counter] = {'Timestamp': new_dataTweet["Timestamp"][i], 
                                 'ID': int(new_dataTweet["ID"][i]), 
                                 'Username': new_dataTweet["Username"][i], 
                                 'Text': new_dataTweet["Text"][i]}     
      counter += 1  

  return new_complaints

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import time

def retrieving_tweet():
  try:
    import_tweet()
    retrieved = lstm_model("jokowi gimana.csv")

  except pd.errors.EmptyDataError:
    time.sleep(5)
    import_tweet()
    retrieved = lstm_model("jokowi gimana.csv")
  
  return retrieved

In [6]:
pip install flask_ngrok

In [ ]:
from flask import Flask, jsonify
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app) 

# import_tweet()

@app.route("/")
def home():
    gatau = retrieving_tweet()
    # gatau = lstm_model("jokowi gimana.csv")
    return jsonify(gatau)
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9c4481ac5af9.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
Status(_api=<tweepy.api.API object at 0x7f92655dd490>, _json={'created_at': 'Mon May 31 08:41:39 +0000 2021', 'id': 1399284928714444803, 'id_str': '1399284928714444803', 'full_text': 'Jalan ceritanya smooth banget, sampe di tengah2 penonton kek dikasih petir di siang bolong😂 plot twist nya gak nahan. Penonton dibikin ngerasa bersalah sejadi2nya WKWKWK', 'truncated': False, 'display_text_range': [0, 169], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []}, 'metadata': {'iso_language_code': 'in', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': 1399284927191928835, 'in_reply_to_status_id_str': '1399284927191928835', 'in_reply_to_user_id': 231076556, 'in_reply_to_user_id_str': '231076556', 'in_reply_to_screen_name': 'farahbdr', 'user': {'id': 231076556, 'id_str': '231076556

127.0.0.1 - - [31/May/2021 09:54:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 09:54:09] "GET /favicon.ico HTTP/1.1" 404 -
